In [1]:
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# Specify the path to the fastai directory
lib_PATH = '/home/ubuntu/fastai/fastai'   # Update this
!cd {lib_PATH}
# Create sym-link to the fastai library
!ln -s {lib_PATH} ./

ln: failed to create symbolic link './fastai': File exists


In [4]:
# Load required modules
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split

# Load Cleaned Data

In [5]:
train = pd.read_feather('/home/ubuntu/wsmd_music_rec/data/tmp/wsdm_train')
test = pd.read_feather('/home/ubuntu/wsmd_music_rec/data/tmp/wsdm_test')

In [6]:
# test.num_plays_by_city.fillna(-1, inplace=True)
# test['num_plays_by_city'] = test['num_plays_by_city'].astype(np.int32)

In [6]:
df, y, nas = proc_df(train, 'target')    # proc_df deals with na values

# Split into Training + Validation Sets

In [ ]:
# def split_vals(a, n): return a[:n].copy(), a[n:].copy()
# https://www.kaggle.com/kamilkk/i-have-to-say-this
# temporally ordered data - set val. set = test set's length

In [12]:
# len_valid = len(test)
# n_trn = len(df) - len_valid
# X_train, X_valid = split_vals(df, n_trn)
# y_train, y_valid = split_vals(y, n_trn)

X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=0.25)

X_train.shape, y_train.shape, X_valid.shape

((5533063, 39), (5533063,), (1844355, 39))

# Train the Random Forest

In [7]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    auc_trn   = roc_auc_score(y_train, m.predict_proba(X_train)[:, 1])
    auc_valid = roc_auc_score(y_valid, m.predict_proba(X_valid)[:, 1])
    res = [auc_trn, auc_valid]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('auc_trn, auc_valid')
    print(res)

In [13]:
# set_rf_samples(1_000_000)
reset_rf_samples()

In [14]:
m = RandomForestClassifier(n_estimators=50, min_samples_leaf=5, n_jobs=-1, verbose=1)
%time m.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.3min


CPU times: user 32min 7s, sys: 10.8 s, total: 32min 18s
Wall time: 2min 22s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.3min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [15]:
print_score(m)

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   12.5s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:   22.7s finished
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    4.1s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    7.5s finished


auc_trn, auc_valid
[0.97278277434231741, 0.79192821525385337]


# Retrain Model Using All Data

In [16]:
def print_score_final(m):
    auc_trn = roc_auc_score(y, m.predict_proba(df)[:, 1])
    res = [auc_trn]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('auc_trn')
    print(res)

In [17]:
# set_rf_samples(1_000_000)
reset_rf_samples()

In [ ]:
m_final = RandomForestClassifier(n_estimators=50, min_samples_leaf=5, n_jobs=-1, verbose=1)
%time m_final.fit(df, y)

In [16]:
print_score_final(m_final)

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   16.7s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:   58.3s finished


auc_trn
[0.99520806957923458]


# Make Predictions

In [12]:
train_cats(test)

In [13]:
test_df, _, _ = proc_df(test, 'id')

In [14]:
%time predictions = m_final.predict_proba(test_df)[:, 1]

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    4.1s


CPU times: user 3min 41s, sys: 1.39 s, total: 3min 42s
Wall time: 15.7 s


[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:   14.9s finished


In [15]:
output = pd.concat([test['id'], pd.DataFrame(predictions)], axis=1)
output.columns = ['id', 'target']   #Changing column names
output.to_csv('predicted.csv', index=False)